In [114]:
import os
WorkHOME = '/Users/Yunjie/Documents/TuneMC'
os.environ['WorkHOME']=WorkHOME
from pythia_functions import MultiProc_Gen
import csv
import numpy as np
from math import sqrt
import json

In [115]:
MultiProc_Gen(7,1428572,WorkHOME)

/Users/Yunjie/Documents/TuneMC/pythia_space/pythia_gen 0 1428572 /Users/Yunjie/Documents/TuneMC/pythia_space/pythia_input.txt /Users/Yunjie/Documents/TuneMC/pythia_space/Output_csv/out_bin_content_0.csv > /Users/Yunjie/Documents/TuneMC/pythia_space/Output_text/Output_0.txt 2>&1
/Users/Yunjie/Documents/TuneMC/pythia_space/pythia_gen 1 1428572 /Users/Yunjie/Documents/TuneMC/pythia_space/pythia_input.txt /Users/Yunjie/Documents/TuneMC/pythia_space/Output_csv/out_bin_content_1.csv > /Users/Yunjie/Documents/TuneMC/pythia_space/Output_text/Output_1.txt 2>&1
/Users/Yunjie/Documents/TuneMC/pythia_space/pythia_gen 2 1428572 /Users/Yunjie/Documents/TuneMC/pythia_space/pythia_input.txt /Users/Yunjie/Documents/TuneMC/pythia_space/Output_csv/out_bin_content_2.csv > /Users/Yunjie/Documents/TuneMC/pythia_space/Output_text/Output_2.txt 2>&1
/Users/Yunjie/Documents/TuneMC/pythia_space/pythia_gen 3 1428572 /Users/Yunjie/Documents/TuneMC/pythia_space/pythia_input.txt /Users/Yunjie/Documents/TuneMC/pythia

In [127]:
def combine_output(n_cores,csv_Dir,bin_widths_block1,bin_widths_block2,outputFile_bin_contents,outputFile_bin_errors):
    input_files = []
    for i_job in range(n_cores):
        input_files.append('{}/out_bin_content_{}.csv'.format(csv_Dir,i_job))

    combined_bin_content = []

    for index,iFile in enumerate(input_files):
        with open(iFile, 'rb') as f:
            reader = csv.reader(f)
            data_list = []
            for row in reader:
                data_list.append([float(i) for i in row])
            data_array = np.array(data_list)
        if index == 0:
            combined_bin_content = data_array.copy()
        else:
            for irow, row in enumerate(combined_bin_content):
                combined_bin_content[irow] = np.add(row,data_array[irow])

    output_bin_contents = []
    output_bin_errors   = []

    N_total   = 0
    N_udsc    = 0
    N_bTagged = 0
    
    #first 10: block1
    for ihist, hist in enumerate(combined_bin_content[0:10]):
        content_row = []
        error_row = []
        row_sum = 0
        
        if ihist == 0:
            for ibin in hist:
                N_total += ibin
                N_udsc  += ibin

        if ihist == 1:
            for ibin in hist:
                N_total   += ibin
                N_bTagged += ibin

        for ibin in range(1,len(hist)-1):
            row_sum += bin_widths_block1[ihist]*hist[ibin]
        
        bin_by_bin_normalize(combined_bin_content[ihist],row_sum,output_bin_contents,output_bin_errors)
    
    #next 6: block2
    print 'N_total   = ', N_total
    print 'N_udsc    = ', N_udsc
    print 'N_bTagged = ', N_bTagged
    
    bin_by_bin_normalize(combined_bin_content[10],float(N_udsc),output_bin_contents,output_bin_errors)
    bin_by_bin_normalize(combined_bin_content[11],float(N_bTagged),output_bin_contents,output_bin_errors)
        
    for ihist,hist in enumerate(combined_bin_content[12:16]):
        row_sum = 0
        
        for ibin in range(1,len(hist)-1):
            row_sum += bin_widths_block2[ihist]*hist[ibin]
        
        bin_by_bin_normalize(combined_bin_content[ihist+12],row_sum,output_bin_contents,output_bin_errors)

    
    #final 4: block3
    nCh_total = float(combined_bin_content[20][0])   
    
    #mesons
    mesons_content_row = []
    mesons_error_row   = []
    for ibin in combined_bin_content[16]:
        mesons_content_row.append(ibin/nCh_total)
        mesons_error_row.append(Get_Ratio_Error_Numbers(ibin,nCh_total))
    
    n_pi    = float(combined_bin_content[16][0])
    n_K     = float(combined_bin_content[16][2])
    n_Kstar = float(combined_bin_content[16][7])
    n_phi   = float(combined_bin_content[16][9])
    
    mesons_content_row.append(n_Kstar/n_K)
    mesons_error_row.append(Get_Ratio_Error_Numbers(n_Kstar,n_K))
    
    mesons_content_row.append(n_phi/n_Kstar)
    mesons_error_row.append(Get_Ratio_Error_Numbers(n_phi,n_Kstar))

    mesons_content_row.append(n_phi/n_K)
    mesons_error_row.append(Get_Ratio_Error_Numbers(n_phi,n_K))

    mesons_content_row.append(n_phi/n_pi)
    mesons_error_row.append(Get_Ratio_Error_Numbers(n_phi,n_pi))
            
    output_bin_contents.append(mesons_content_row)
    output_bin_errors.append(mesons_error_row)
    
    #baryons
    baryons_content_row = []
    baryons_error_row   = []

    for ibin in combined_bin_content[17]:
        baryons_content_row.append(ibin/nCh_total)
        baryons_error_row.append(Get_Ratio_Error_Numbers(ibin,nCh_total))

    n_p      = float(combined_bin_content[17][0])
    n_Lambda = float(combined_bin_content[17][1])
    
    baryons_content_row.insert(2,n_Lambda/n_p)
    baryons_error_row.insert(2,Get_Ratio_Error_Numbers(n_Lambda,n_p))
    
    baryons_content_row.insert(3,n_Lambda/n_K)
    baryons_error_row.insert(3,Get_Ratio_Error_Numbers(n_Lambda,n_K))

    output_bin_contents.append(baryons_content_row)
    output_bin_errors.append(baryons_error_row)
    
    #charm and beauty
    combined_bin_content[19][7] *= 10
    for ihist,hist in enumerate(combined_bin_content[18:20]):
        bin_by_bin_normalize_rates(combined_bin_content[ihist+18],float(N_total),output_bin_contents,output_bin_errors)
    
    
    content_file = open(outputFile_bin_contents,'w')
    content_file_writer = csv.writer(content_file,delimiter =',',
                                     quotechar=" ",lineterminator="\n",quoting=csv.QUOTE_ALL)
    error_file = open(outputFile_bin_errors,'w')
    error_file_writer = csv.writer(error_file,delimiter =',',
                                   quotechar=" ",lineterminator="\n",quoting=csv.QUOTE_ALL)

    for ihist in range(len(output_bin_contents)):
        content_file_writer.writerow(output_bin_contents[ihist])
        error_file_writer.writerow(output_bin_errors[ihist])

    content_file.close()
    error_file.close()

In [118]:
def bin_by_bin_normalize(hist,norm,output_content,output_error):
    content_row = []
    error_row   = []

    for ibin in range(1,len(hist)-1):
        content = hist[ibin]/norm
        content_row.append(content)
        error = Get_Ratio_Error_Numbers(hist[ibin],norm)
        error_row.append(error) 

    output_content.append(content_row)
    output_error.append(error_row)

def bin_by_bin_normalize_rates(hist,norm,output_content,output_error):
    content_row = []
    error_row   = []

    for ibin in range(len(hist)):
        content = hist[ibin]/norm
        content_row.append(content)
        error = Get_Ratio_Error_Numbers(hist[ibin],norm)
        error_row.append(error) 

    output_content.append(content_row)
    output_error.append(error_row)

In [119]:
def Get_Ratio_Error_Numbers(num,denom):
    num=float(num)
    denom=float(denom)
    value = num/denom
    error = 0.
    if num == 0 and denom != 0:
        error = value * sqrt(1./denom)
    else:
        error = value * sqrt(1./num + 1./denom)
    return error

In [120]:
def get_chi2(blocks,object_contents,object_errors,tune_contents,tune_errors,output_file):
    
    input_files = [object_contents,object_errors,tune_contents,tune_errors]

    lists = []

    for index,iFile in enumerate(input_files):
        with open(iFile, 'rb') as f:
            reader = csv.reader(f)
            data_list = []
            for row in reader:
                data_list.append([float(i) for i in row])
            lists.append(np.array(data_list))

    chi2s    = []

    ofile = open(output_file,'a')
    ofile_writer = csv.writer(ofile,delimiter =',',
                              quotechar=" ",lineterminator="\n",quoting=csv.QUOTE_ALL)

    for ihist in range(len(lists[0])):

        ihist_chi2 = 0

        for ibin in range(len(lists[0][ihist])):
            o_value = lists[0][ihist][ibin]
            o_error = lists[1][ihist][ibin]
            t_value = lists[2][ihist][ibin]
            t_error = lists[3][ihist][ibin]

            o_error = check_one_percent(o_error,o_value)
            t_error = check_one_percent(t_error,t_value)

            chi2 = compute_chi2(o_value,o_error,t_value,t_error)

            ihist_chi2 += chi2

        chi2s.append(ihist_chi2)
        
        
    result_chi2 = 0.
    for index,chi2 in enumerate(chi2s):
        if blocks[0] and index >= 0  and index < 10:
            result_chi2 += chi2
        if blocks[1] and index >= 10 and index < 16:
            result_chi2 += chi2
        if blocks[2] and index >= 16 and index < 19:
            result_chi2 += chi2
        
    
        
    ofile_writer.writerow(chi2s)
    ofile.close()

    return result_chi2

In [121]:
def load_config(config_file):
    try:
        with open(config_file,'r') as json_file:
            json_data = json.load(json_file)
            return json_data
    except:
        raise Exception('json file loading failed. Please check commas or parentheses?')

def str_to_bool(s):
    if s == 'True':
         return True
    elif s == 'False':
         return False
    else:
         raise ValueError("Cannot covert string {} to a bool.".format(s))
            
def check_one_percent(error,value):

    if value != 0.:
        if error/value <= 0.01:
            return value*0.01
        else:
            return error
    else:
        return error

def compute_chi2(o_value,o_error,t_value,t_error):
    error2 = o_error**2 + t_error**2
    if error2 != 0.:
        return (o_value-t_value)**2/error2
    else:
        return 0.

In [122]:
bin_widths_block1 = [0.025,0.025,0.05,0.05,0.032,0.032,0.015,0.015,0.02,0.02]
bin_widths_block2 = [0.2,0.2,0.05,0.05]

In [49]:
Alist=[1,23,4,56]
Alist.insert(1,2)
print Alist

[1, 2, 23, 4, 56]


In [123]:
csv_Dir = '{}/pythia_space/Output_csv'.format(WorkHOME)
outputFile_bin_contents = '{}/output_bin_contents.csv'.format(csv_Dir)
outputFile_bin_errors   = '{}/output_bin_errors.csv'.format(csv_Dir)

object_contents = '{}/output_bin_contents_Monash.csv'.format(csv_Dir)
object_errors   = '{}/output_bin_errors_Monash.csv'.format(csv_Dir)

tune_contents   = '{}/output_bin_contents.csv'.format(csv_Dir)
tune_errors     = '{}/output_bin_errors.csv'.format(csv_Dir)

obj_file        = '{}/interface/objectives.csv'.format(WorkHOME)

In [52]:
alist=range(22)
print alist[18:20]

[18, 19]


In [128]:
combine_output(7,csv_Dir,bin_widths_block1,bin_widths_block2,outputFile_bin_contents,outputFile_bin_errors)

N_total   =  10000002.0
N_udsc    =  7793976.0
N_bTagged =  2206026.0


In [133]:
config=load_config('{}/tune_config.json'.format(WorkHOME))

blocks = []
for i in range(1,4):
    blocks.append(str_to_bool(config['block{}'.format(i)]))

print blocks

if os.path.exists(obj_file):
    os.remove(obj_file)

chi2 = get_chi2(blocks,object_contents,object_errors,tune_contents,tune_errors,obj_file)
print chi2

[False, True, False]
107.009514163


In [134]:
    spectra_names_block1  =  ["hThrusts_udsc", "hThrusts_b",
                              "hC_param_udsc", "hC_param_b",
                              "hD_param_udsc", "hD_param_b",
                              "hB_W_udsc","hB_W_b",
                              "hB_T_udsc","hB_T_b"]

    spectra_names_block2  =  ["hChargeMulti_udsc","hChargeMulti_b",
                              "hMomentFrac_udsc","hMomentFrac_b",
                              "hMomentFrac_Dstar","hMomentFrac_bWeak"]

    spectra_names_block3  =  ["hMesonFrac","hBaryonFrac",
                              "hCharmRates","hBeautyRates"]

    spectra = [spectra_names_block1,spectra_names_block2,spectra_names_block3]

    spectra_names = []

    for index,block in enumerate(blocks):
        if block:
            spectra_names += spectra[index]

In [135]:
print spectra_names

['hChargeMulti_udsc', 'hChargeMulti_b', 'hMomentFrac_udsc', 'hMomentFrac_b', 'hMomentFrac_Dstar', 'hMomentFrac_bWeak']


In [173]:
Spearmint_output_FILE='{}/interface/spearmint_output.txt'.format(WorkHOME)

param_index = 0
inFile = open(Spearmint_output_FILE)
lines = list(inFile)
inFile.seek(0)


def find_order():
    
    param_order = []
    with open('{}/pythia_space/pars_to_tune.txt'.format(WorkHOME),'r') as parsList:
        param_names = parsList.read().splitlines()
        
    print param_names
    
    found = False
    index = 0
    
    while not found:
        line = lines[index]
        lo_index = line.find("Minimum expected objective value under model is")
        str_len  = len("Minimum expected objective value under model is")
        up_index = line.find("(+/-")
        
        if lo_index != -1 and up_index !=-1:
                current_param_index = index+3
                value_start_index=lines[current_param_index-1].find("----")
                value_end_index = value_start_index+13
                for ind,line in enumerate(lines[current_param_index:current_param_index+len(param_names)]): 
                    string = str(line[value_start_index:value_end_index])
                    string = string.replace(" ","")
                    for param in param_names:
                        if string in param:
                            param_order.append(param)
                found = True
                    
        index += 1
        
    return param_order

In [181]:
all_pars_dict = load_config('{}/utils/all_pars_dict.json'.format(WorkHOME))

def get_MonashValues():
    with open('{}/pythia_space/pars_to_tune.txt'.format(WorkHOME),'r') as parsList:
        param_names = parsList.read().splitlines()
        
    values =[]
    
    for par in param_names:
        values.append(all_pars_dict[par]['Monash'])
        
    return values

def get_ranges():
    with open('{}/pythia_space/pars_to_tune.txt'.format(WorkHOME),'r') as parsList:
        param_names = parsList.read().splitlines()
    
    param_ranges = []
    for par in param_names:
        param_ranges.append((all_pars_dict[par]['min'],all_pars_dict[par]['max']))
    return param_ranges

In [182]:
param_order = find_order()
print param_order
MonashValues = get_MonashValues()
print MonashValues
param_ranges = get_ranges()
print param_ranges

['rFactC', 'bLund', 'rFactB', 'aExtraSQuark', 'sigma', 'aExtraDiquark']
16
29
33
['rFactB', 'rFactC', 'bLund', 'aExtraSQuark', 'sigma', 'aExtraDiquark']
[1.32, 0.98, 0.855, 0.0, 0.335, 0.97]
[(0.0, 2.0), (0.2, 2.0), (0.0, 2.0), (0.0, 2.0), (0.0, 1.0), (0.0, 2.0)]


In [185]:
blocks = [False,True,False]

def nRows_nCols(blocks):
    if blocks == [True,False,False]:
        return 3,1,5,2

    elif blocks == [False,True,False]:
        return 3,2,3,2
    
    elif blocks == [False,False,True]:
        return 4,3,2,2
    else:
        return 5,4,5,4
    
param_nRows,param_nCols,spectra_nRows,spectra_nCols = nRows_nCols(blocks)
print param_nRows,param_nCols,spectra_nRows,spectra_nCols



3 2 3 2
